In [ ]:
import numpy as np
import qutip as qt

# generate ideal targets
qCZ = qt.cphase(np.pi)
qRX90 = (-np.pi*.25j*qt.sigmax()).expm()
qI = qt.qeye(2)

In [ ]:
import grove.tomography.qpu_characterization
reload(grove.tomography.qpu_characterization);

In [ ]:
from pyquil.api import get_devices, QPUConnection, QVMConnection
from pyquil.gates import RX, CZ, I
from grove.tomography.qpu_characterization import parallel_process_tomographies, parallel_sample_assignment_probs
from grove.tomography.process_tomography import DEFAULT_PROCESS_TOMO_SETTINGS

import grove.tomography.utils as ut

# fancy TQDM progress bars
ut.notebook_mode(True)

In [ ]:
DEFAULT_PROCESS_TOMO_SETTINGS

In [45]:
get_devices()

{<Device 19Q-Acorn retuning>, <Device 8Q-Agave offline>}

In [ ]:
qpu = QPUConnection('8Q-Agave', status_time=False)
# qpu = QVMConnection()

In [ ]:
%%time
I_tomos = parallel_process_tomographies(I, [[q] for q in range(8)], 2000, qpu)
RX90_tomos = parallel_process_tomographies(RX(np.pi/2), [[q] for q in range(8)], 2000, qpu)

In [ ]:
staggered_1q1 = [[q] for q in range(0, 8, 2)]
staggered_1q2 = [[q] for q in range(1, 8, 2)]

In [ ]:
neighboring_cliques1 = [(0, 1), (2, 3), (4, 5), (6, 7)]
neighboring_cliques2 = [(7, 0), (1, 2), (3, 4), (5, 6)]

staggered_cliques1 = [(0, 1), (4, 5)]
staggered_cliques2 = [(2, 3), (6, 7)]
staggered_cliques3 = [(7, 0), (3, 4)]
staggered_cliques4 = [(1, 2), (5, 6)]

In [ ]:
%%time
CZ_tomos1 = parallel_process_tomographies(CZ, neighboring_cliques1, 2000, qpu)

In [ ]:
%%time
CZ_tomos2 = parallel_process_tomographies(CZ, neighboring_cliques2, 2000, qpu)

In [ ]:
%%time 
RX90_tomos_staggered1 = parallel_process_tomographies(RX(np.pi/2), staggered_1q1, 2000, qpu)
RX90_tomos_staggered2 = parallel_process_tomographies(RX(np.pi/2), staggered_1q2, 2000, qpu)

In [ ]:
%%time
CZ_tomos_staggered1 = parallel_process_tomographies(CZ, staggered_cliques1, 2000, qpu)
CZ_tomos_staggered2 = parallel_process_tomographies(CZ, staggered_cliques2, 2000, qpu)

In [ ]:
%%time
CZ_tomos_staggered3 = parallel_process_tomographies(CZ, staggered_cliques3, 2000, qpu)
CZ_tomos_staggered4 = parallel_process_tomographies(CZ, staggered_cliques4, 2000, qpu)

In [ ]:
import cPickle as pickle
with open("qpu_gates_constrained.pickle", "wb") as f:
    tomos = {k: v for k,v in globals().items() if "_tomos" in k}
    print("Storing {}".format(list(tomos.keys())))
    pickle.dump(tomos, f)

In [ ]:
import qutip as qt

In [43]:
print("Simultaneously taken on {}\n".format(neighboring_cliques1))
for pair, pt in zip(neighboring_cliques1, CZ_tomos1[0]):
    print("Average Gate Fidelity for CZ {} F_avg={:0.3g}%".format(pair, 100*pt.avg_gate_fidelity(qCZ)))
print("\nSimultaneously taken on {}\n".format(neighboring_cliques2))
for pair, pt in zip(neighboring_cliques2, CZ_tomos2[0]):
    print("Average Gate Fidelity for CZ {} F_avg={:0.3g}%".format(pair, 100*pt.avg_gate_fidelity(qCZ)))

Simultaneously taken on [(0, 1), (2, 3), (4, 5), (6, 7)]

Average Gate Fidelity for CZ (0, 1) F_avg=73.6%
Average Gate Fidelity for CZ (2, 3) F_avg=71.9%
Average Gate Fidelity for CZ (4, 5) F_avg=75.2%
Average Gate Fidelity for CZ (6, 7) F_avg=53%

Simultaneously taken on [(7, 0), (1, 2), (3, 4), (5, 6)]

Average Gate Fidelity for CZ (7, 0) F_avg=62.7%
Average Gate Fidelity for CZ (1, 2) F_avg=26.2%
Average Gate Fidelity for CZ (3, 4) F_avg=71%
Average Gate Fidelity for CZ (5, 6) F_avg=65.7%


In [ ]:
print("Simultaneously taken on {}\n".format(neighboring_cliques1))
for pair, pt in zip(neighboring_cliques1, CZ_tomos1[0]):
    print("Average Gate Fidelity for CZ {} F_avg={:0.3g}%".format(pair, 100*pt.avg_gate_fidelity(qCZ)))
print("\nSimultaneously taken on {}\n".format(neighboring_cliques2))
for pair, pt in zip(neighboring_cliques2, CZ_tomos2[0]):
    print("Average Gate Fidelity for CZ {} F_avg={:0.3g}%".format(pair, 100*pt.avg_gate_fidelity(qCZ))) 

In [ ]:
for pt in CZ_tomos2[0]:
    print(pt.avg_gate_fidelity(qCZ))

In [44]:
print("Simultaneously taken on {}\n".format(staggered_cliques1))
for pair, pt in zip(staggered_cliques1, CZ_tomos_staggered1[0]):
    print("Average Gate Fidelity for CZ {} F_avg={:0.3g}%".format(pair, 100*pt.avg_gate_fidelity(qCZ)))
print("\nSimultaneously taken on {}\n".format(staggered_cliques2))
for pair, pt in zip(staggered_cliques2, CZ_tomos_staggered2[0]):
    print("Average Gate Fidelity for CZ {} F_avg={:0.3g}%".format(pair, 100*pt.avg_gate_fidelity(qCZ))) 

Simultaneously taken on [(0, 1), (4, 5)]

Average Gate Fidelity for CZ (0, 1) F_avg=91.5%
Average Gate Fidelity for CZ (4, 5) F_avg=90.1%

Simultaneously taken on [(2, 3), (6, 7)]

Average Gate Fidelity for CZ (2, 3) F_avg=73.1%
Average Gate Fidelity for CZ (6, 7) F_avg=84.1%


In [ ]:
for pt in CZ_tomos_staggered1[0]:
    print(pt.avg_gate_fidelity(qCZ))

In [ ]:
for pt in RX90_tomos[0]:
    print(pt.avg_gate_fidelity((-qt.sigmax()*np.pi*.25j).expm()))

In [ ]:
CZ_tomos_staggered1[0][0].plot();